In [13]:
#This is an example that uses the websockets api to know when a prompt execution is done
#Once the prompt execution is done it downloads the images using the /history endpoint

import websocket #NOTE: websocket-client (https://github.com/websocket-client/websocket-client)
import uuid
import json
import urllib.request
import urllib.parse

server_address = "100.109.232.60:8188"
client_id = str(uuid.uuid4())

def queue_prompt(prompt):
    p = {"prompt": prompt, "client_id": client_id}
    data = json.dumps(p).encode('utf-8')
    req =  urllib.request.Request("http://{}/prompt".format(server_address), data=data)
    return json.loads(urllib.request.urlopen(req).read())

def get_image(filename, subfolder, folder_type):
    data = {"filename": filename, "subfolder": subfolder, "type": folder_type}
    url_values = urllib.parse.urlencode(data)
    with urllib.request.urlopen("http://{}/view?{}".format(server_address, url_values)) as response:
        return response.read()

def get_history(prompt_id):
    with urllib.request.urlopen("http://{}/history/{}".format(server_address, prompt_id)) as response:
        return json.loads(response.read())

def get_images(ws, prompt):
    prompt_id = queue_prompt(prompt)['prompt_id']
    output_images = {}
    while True:
        out = ws.recv()
        if isinstance(out, str):
            message = json.loads(out)
            if message['type'] == 'executing':
                data = message['data']
                if data['node'] is None and data['prompt_id'] == prompt_id:
                    break #Execution is done
        else:
            continue #previews are binary data

    history = get_history(prompt_id)[prompt_id]['outputs']
    return history
    for o in history['outputs']:
        for node_id in history['outputs']:
            node_output = history['outputs'][node_id]
            if 'images' in node_output:
                images_output = []
                for image in node_output['images']:
                    image_data = get_image(image['filename'], image['subfolder'], image['type'])
                    images_output.append(image_data)
            output_images[node_id] = images_output

    return output_images

In [14]:
prompt_text = """
{
  "115": {
    "inputs": {
      "title": "Input Image",
      "short_description": "",
      "subtype": "image",
      "required": true,
      "value": "https://ireland.apollo.olxcdn.com/v1/files/eyJmbiI6InhocWZpbmszOTgxMzEtU1REVlRMUFQifQ.UFAMZqIpQuwrKhiZNJgVouBe6m6XI0pbmmGYvmt-i1w/image;s=644x461"
    },
    "class_type": "signature_input_image",
    "_meta": {
      "title": "Input Image"
    }
  },
  "117": {
    "inputs": {
      "model_name": "rmbg14",
      "image": [
        "115",
        0
      ]
    },
    "class_type": "Background Removal",
    "_meta": {
      "title": "Background Removal"
    }
  },
  "119": {
    "inputs": {
      "title": "Output Image",
      "short_description": "",
      "subtype": "image",
      "value": [
        "117",
        0
      ]
    },
    "class_type": "signature_output",
    "_meta": {
      "title": "Output"
    }
  },
  "133": {
    "inputs": {
      "title": "Positive Prompt",
      "short_description": "",
      "subtype": "positive_prompt",
      "required": true,
      "value": "cenas muita fixes"
    },
    "class_type": "signature_input_text",
    "_meta": {
      "title": "Input Text"
    }
  },
  "134": {
    "inputs": {
      "title": "Negative Prompt",
      "short_description": "",
      "subtype": "negative_prompt",
      "required": true,
      "value": "estou com fome esta quase na hora de almoco"
    },
    "class_type": "signature_input_text",
    "_meta": {
      "title": "Input Text"
    }
  },
  "135": {
    "inputs": {
      "title": "Output Mask",
      "short_description": "",
      "subtype": "mask",
      "value": [
        "117",
        1
      ]
    },
    "class_type": "signature_output",
    "_meta": {
      "title": "Output"
    }
  }
}
"""

In [15]:
prompt = json.loads(prompt_text)
ws = websocket.WebSocket()
ws.connect("ws://{}/ws?clientId={}".format(server_address, client_id))
images = get_images(ws, prompt)

print(images)

{'119': {'signature_output': [{'title': 'Output Image', 'short_description': '', 'type': 'image', 'value': 'signature_20240318_173341_56231.png'}]}, '135': {'signature_output': [{'title': 'Output Mask', 'short_description': '', 'type': 'image', 'value': 'signature_20240318_173341_86870.png'}]}}
